In [36]:
import os.path
from typing import List

import numpy as np
import pandas as pd
from pandas import DataFrame, Series

from datetime import date
from thefuzz import fuzz

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_colwidth = 160

from utils.configuration import settings
from utils.utils import *
from tft.api import *

In [37]:
API_KEY: str = settings.api_key
ASSETS_DIR: str = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers' # challengers grandmasters
MAX_COUNT: int = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344
PATCH: date = date(2022, 7, 1)

TARGETNAME = 'placement'

# Data Loading

In [38]:
raw_df: DataFrame = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.pickle'))

# Preprocessing

In [39]:
def impute(df) -> DataFrame:
    for name in df.select_dtypes("number"):
        df[name] = df[name].fillna(0)
    for name in df.select_dtypes("object"):
        df[name] = df[name].fillna("None")
    return df

raw_df = impute(raw_df)

In [40]:
match_id_df: Series = raw_df['match_id']
X: DataFrame = raw_df.drop(['match_id'], axis=1)
y: Series = X.pop(TARGETNAME)
X.fillna('', inplace=True)
numeric_cols: List = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(include=['object','category']).columns.tolist()
traits_col: list = [s for s in numeric_cols if "Set7" in s]
units_col: list = [s for s in numeric_cols if "TFT7" in s]
augments_col: list[str] = ['augment0', 'augment1', 'augment2']
items_col = [s for s in categorical_cols if s not in augments_col]
df_unique = X.nunique().to_frame().reset_index()
df_unique.columns = ['Variable','DistinctCount']
unique_items_set = {y for col in items_col for y in X[col].unique().tolist()}
unique_augments_set = {y for col in augments_col for y in X[col].unique().tolist()}
X[f'items_count'] = X[items_col].apply(lambda row: sum(x != 'None' for x in row), axis=1)
X[f'traits_sum'] = X[traits_col].sum(axis=1)
X[f'units_sum'] = X[units_col].sum(axis=1)
X.iloc[X[f'units_sum'].idxmax()]

augment0              TFT7_Augment_MageConference
augment1                   TFT7_Augment_ThinkFast
augment2          TFT6_Augment_CelestialBlessing1
Set7_Assassin                                 0.0
Set7_Astral                                   1.0
                               ...               
TFT7_Zoe_item1                               None
TFT7_Zoe_item2                               None
items_count                                    10
traits_sum                                    8.0
units_sum                                    29.0
Name: 5823, Length: 270, dtype: object

In [41]:
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(include=['object','category']).columns.tolist()
X[numeric_cols] = X[numeric_cols].applymap(np.int64)
matches_df = X.copy()
matches_df[TARGETNAME] = y

In [42]:
def get_unit_items_ranking(df: matches_df, unit: str):
    # filter and melt the dataframe
    df = df.filter(regex=f'placement|{unit}_item0|{unit}_item1|{unit}_item2')
    df[f'unit'] = f'{unit}' # fill in current unit
    # join 3 items to 1 column
    df[f'{unit}_items'] = df[[f'{unit}_item0', f'{unit}_item1', f'{unit}_item2']].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)
    # sort items for unique combination
    df[f'{unit}_items'] = df[f'{unit}_items'].apply(lambda x: ', '.join(sorted(x.split(', '))))
    df = df.filter(regex=f'placement|{unit}_items|unit')
    m = df.melt(
        ['placement',f'unit'], value_name=f'{unit}_items_grp') #, value_vars=[f'{unit}_items', f'{unit}']
    # group and aggregate mean/median average_placement
    dct = {'value_count': (f'{unit}_items_grp', 'count'),
           'average_placement': ('placement', 'mean')}
    return m.groupby([f'unit', f'{unit}_items_grp'], as_index=False).agg(**dct).sort_values(by='average_placement')

In [43]:
get_unit_items_ranking(df = matches_df[:2], unit='TFT7_Zoe')

,unit,TFT7_Zoe_items_grp,value_count,average_placement
0,TFT7_Zoe,"None, None, None",2,1.5


In [44]:
# Get top5 
top5_items_list = []
for unit in units_col:
    df = get_unit_items_ranking(df = matches_df, unit=unit)
    df = df[df['value_count']>=12][:5] #Top 5 with counts >= 12
    top5_items_list.extend(df.values)

In [45]:
top5_items_list = pd.DataFrame(top5_items_list, columns=['unit', 'items',	'value_count',	'average_placement'])

In [46]:
top5_items_list.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_top5_items.csv'), index=False)

In [47]:
top5_items_list.groupby('unit').head(1)

,unit,items,value_count,average_placement
0,TFT7_Aatrox,"None, None, TitanicHydra",12,3.000000
3,TFT7_Anivia,"ArchangelsStaff, ArchangelsStaff, Morellonomicon",13,3.076923
8,TFT7_AoShin,"ArchangelsStaff, HextechGunblade, SpearOfShojin",44,3.318182
13,TFT7_Ashe,"None, None, SparringGloves",14,3.142857
18,TFT7_AurelionSol,"None, None, None",23966,4.498748
19,TFT7_Bard,"None, None, Redemption",13,2.153846
24,TFT7_Braum,"ChainVest, None, None",13,3.923077
29,TFT7_Corki,"GuinsoosRageblade, InfinityEdge, RunaansHurricane",14,3.428571
34,TFT7_Diana,"FrozenHeart, IonicSpark, RedBuff",41,3.804878
39,TFT7_DragonBlue,"GuinsoosRageblade, Quicksilver, RevelEmblemItem",33,2.757576


# Load TFT asset

In [48]:
tft_assets = read_json(os.path.join(ASSETS_DIR, f'en_us.json'))

In [49]:
tft7_set = tft_assets['setData'][6]['champions'] #['apiName'] ['traits']['name'] #['champions'] #['champions'].name

In [50]:
champions_dict = {}

for champion in tft7_set:
    if champion["apiName"] not in champions_dict:
        champions_dict[champion["apiName"]] = []
    for trait in champion["traits"]:
        champions_dict[champion["apiName"]].append(trait)

In [51]:
champions_dict['TFT7_Olaf']

['Scalescorn', 'Bruiser', 'Warrior']

# Team Composition Ranking

In [52]:
def add_traits(units_str):
    # for units in units_str.split(', '):
    comp_array = []
    if len(units_str) == 0:
        return ''
    for unit in units_str.split(', '):
        traits_array = []
        for trait in champions_dict[unit]:
            traits_array.append(trait[:2]+trait[-1:]) # Add first 2 char for trait
        traits_str = '-'.join(traits_array) + f'-{unit}'
        comp_array.append(traits_str)

    # print(f'{"".join(comp_array)}')
    return ','.join(comp_array)

def get_unit_comp_ranking(df: matches_df):
    # filter and melt the dataframe
    df = df.filter(['placement']+units_col)
    # join units lvl > 0 to 1 column
    df['comp'] = df[units_col].apply(lambda row: ', '.join(row[row > 0].index.values.astype(str)), axis=1)
    df['comp'] = df['comp'].apply(add_traits)

    df['comp'] = df['comp'].str.replace('TFT7_','') # remove prefix .split('_',1).str[-1]
    df = df.filter(['placement', 'comp'])
    m = df.melt(
        ['placement'], value_name=f'comp_grp')
    # group and aggregate mean/median average_placement
    dct = {'value_count': (f'comp_grp', 'count'),
           'average_placement': ('placement', 'mean')}
    return m.groupby([f'comp_grp'], as_index=False).agg(**dct).sort_values(by='average_placement')

In [53]:
# Get top5 
comp_df = get_unit_comp_ranking(df = matches_df)

In [54]:
top5_comp_list = []
m = comp_df[comp_df['value_count']>=1] #[:5] #Top 5 with counts >= 12
top5_comp_list.extend(m.values)
comp_ranking_df = pd.DataFrame(top5_comp_list, columns=['comp', 'value_count', 'average_placement'])

In [55]:
comp_ranking_df #.groupby('comp').head(1)

,comp,value_count,average_placement
0,"Whs-Brr-Drn-DragonPurple,Whs-Shr-Elise,Jae-Shr-Neeko,Whs-Asn-Pyke,Jae-Str-Soraka,Whs-Mae-Brr-Sylas,Jae-Gun-Taric,Mie-Drr-War-Yasuo",1,1.0
1,"Gud-Myc-Bad-Bard,Whs-Brr-Drn-DragonPurple,Tet-Brr-Led-Ornn,Whs-Asn-Pyke,Tet-Asn-Qiyana,Jae-Str-Soraka,Gud-Swt-Twitch,Mie-Drr-War-Yasuo",1,1.0
2,"Gud-Myc-Bad-Bard,Whs-Brr-Drn-DragonPurple,Tet-Brr-Led-Ornn,Whs-Asn-Pyke,Tet-Asn-Qiyana,Jae-Str-Soraka,Gud-Asn-Talon,Mie-Drr-War-Yasuo",2,1.0
3,"Gud-Myc-Bad-Bard,Whs-Brr-Drn-DragonPurple,Tet-Brr-Led-Ornn,Whs-Asn-Pyke,Tet-Asn-Qiyana,Gud-Asn-Talon,Whs-Gun-Thresh,Mie-Drr-War-Yasuo",1,1.0
4,"Mie-Drn-DragonBlue,Trr-Myc-Evr-Lulu,Tet-Brr-Led-Ornn,Rel-Evr-Sona,Whs-Mae-Brr-Sylas,Mie-Drr-War-Yasuo,She-Spf-Mae-Zoe",1,1.0
...,...,...,...
11953,"Scn-Asn-Diana,Whs-Brr-Drn-DragonPurple,Asl-Brr-Illaoi,Tet-Asn-Qiyana,Rag-Brr-War-Shen,Whs-Mae-Brr-Sylas",1,8.0
11954,"Scn-Asn-Diana,Whs-Brr-Drn-DragonPurple,Asl-Brr-Illaoi,Tet-Brr-Led-Ornn,Tet-Asn-Qiyana,Rag-Brr-War-Shen,Gud-Asn-Talon",1,8.0
11955,"Scn-Asn-Diana,Whs-Brr-Drn-DragonPurple,Asl-Brr-Illaoi,Whs-Asn-Pyke,Rag-Brr-War-Shen,Whs-Mae-Brr-Sylas,Rel-Brr-TahmKench",1,8.0
11956,"Scn-Gun-Braum,Scn-Asn-Diana,Scn-Car-Mae-Lillia,Asl-Mae-Myc-Nami,Mie-Car-Nunu,Scn-Brr-War-Olaf,Gud-Mae-Ryze,Asl-Brr-Skarner,Asl-Mae-Vladimir",1,8.0


# Team composition Clustering

In [56]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN

embedder = SentenceTransformer('all-MiniLM-L6-v2')

--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\furyx\miniconda3\envs\tft\lib\logging\__init__.py", line 1104, in emit
    self.flush()
  File "c:\Users\furyx\miniconda3\envs\tft\lib\logging\__init__.py", line 1084, in flush
    self.stream.flush()
OSError: [Errno 22] Invalid argument
Call stack:
  File "c:\Users\furyx\miniconda3\envs\tft\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\furyx\miniconda3\envs\tft\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\furyx\miniconda3\envs\tft\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\furyx\miniconda3\envs\tft\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
    app.start()
  File "c:\Users\furyx\miniconda3\envs\tft\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
    self.io_loop.start()
  File "c:\Users\furyx\m

In [57]:
corpus_sentences = list(comp_ranking_df['comp'])

corpus_embeddings = embedder.encode(corpus_sentences, batch_size=512, show_progress_bar=True, convert_to_tensor=True)
# Normalize the embeddings to unit length
# corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

Batches: 100%|██████████| 24/24 [03:17<00:00,  8.21s/it]


In [58]:
clustering_model = DBSCAN(eps=0.023, min_samples=1, metric='cosine', n_jobs=-1)
predict=clustering_model.fit_predict(corpus_embeddings)
comp_ranking_df['group'] = pd.Series(predict, index=comp_ranking_df.index)

In [59]:
comp_ranking_df.sort_values(by='group')

,comp,value_count,average_placement,group
0,"Whs-Brr-Drn-DragonPurple,Whs-Shr-Elise,Jae-Shr-Neeko,Whs-Asn-Pyke,Jae-Str-Soraka,Whs-Mae-Brr-Sylas,Jae-Gun-Taric,Mie-Drr-War-Yasuo",1,1.0,0
10475,"Whs-Brr-Drn-DragonPurple,Rag-Car-Hecarim,Tet-Brr-Led-Ornn,Whs-Asn-Pyke,Tet-Asn-Qiyana,Rag-Brr-War-Shen,Whs-Mae-Brr-Sylas",1,8.0,0
10472,"Whs-Brr-Drn-DragonPurple,Rag-Car-Hecarim,Tet-Drr-LeeSin,Tet-Brr-Led-Ornn,Gud-Car-Sejuani,Rag-Brr-War-Shen,Whs-Mae-Brr-Sylas",1,8.0,0
10471,"Whs-Brr-Drn-DragonPurple,Whs-Shr-Elise,Jae-Shr-Gnar,Jae-Shr-Neeko,Asl-Shr-Nidalee,Tet-Asn-Qiyana",1,8.0,0
964,"Whs-Brr-Drn-DragonPurple,Scn-Brr-War-Olaf,Tet-Brr-Led-Ornn,Whs-Asn-Pyke,Tet-Asn-Qiyana,Rag-Brr-War-Shen,Whs-Mae-Brr-Sylas,Rel-Brr-TahmKench,Whs-Gun-Thresh",1,1.0,0
...,...,...,...,...
11947,"Scn-Asn-Diana,She-Drn-Gun-DragonGold,Trr-Myc-Evr-Lulu,Scn-Brr-War-Olaf,Tet-Brr-Led-Ornn,Tet-Asn-Qiyana,Rel-Evr-Sona",1,8.0,3136
11950,"Scn-Asn-Diana,Tet-Swt-Ezreal,Scn-Car-Mae-Lillia,Tet-Asn-Qiyana,Gud-Car-Sejuani,Gud-Swt-Twitch",1,8.0,3137
11955,"Scn-Asn-Diana,Whs-Brr-Drn-DragonPurple,Asl-Brr-Illaoi,Whs-Asn-Pyke,Rag-Brr-War-Shen,Whs-Mae-Brr-Sylas,Rel-Brr-TahmKench",1,8.0,3138
11956,"Scn-Gun-Braum,Scn-Asn-Diana,Scn-Car-Mae-Lillia,Asl-Mae-Myc-Nami,Mie-Car-Nunu,Scn-Brr-War-Olaf,Gud-Mae-Ryze,Asl-Brr-Skarner,Asl-Mae-Vladimir",1,8.0,3139


In [60]:
comp_ranking_df['grp_count'] = comp_ranking_df.groupby(['group'], as_index=False)['value_count'].transform('sum')
comp_ranking_df['grp_placement'] = comp_ranking_df.groupby(['group'], as_index=False)['average_placement'].transform('mean')

In [61]:
comp_ranking_df.sort_values(by='group')[:60]

,comp,value_count,average_placement,group,grp_count,grp_placement
0,"Whs-Brr-Drn-DragonPurple,Whs-Shr-Elise,Jae-Shr-Neeko,Whs-Asn-Pyke,Jae-Str-Soraka,Whs-Mae-Brr-Sylas,Jae-Gun-Taric,Mie-Drr-War-Yasuo",1,1.000000,0,2585,4.71774
10475,"Whs-Brr-Drn-DragonPurple,Rag-Car-Hecarim,Tet-Brr-Led-Ornn,Whs-Asn-Pyke,Tet-Asn-Qiyana,Rag-Brr-War-Shen,Whs-Mae-Brr-Sylas",1,8.000000,0,2585,4.71774
10472,"Whs-Brr-Drn-DragonPurple,Rag-Car-Hecarim,Tet-Drr-LeeSin,Tet-Brr-Led-Ornn,Gud-Car-Sejuani,Rag-Brr-War-Shen,Whs-Mae-Brr-Sylas",1,8.000000,0,2585,4.71774
10471,"Whs-Brr-Drn-DragonPurple,Whs-Shr-Elise,Jae-Shr-Gnar,Jae-Shr-Neeko,Asl-Shr-Nidalee,Tet-Asn-Qiyana",1,8.000000,0,2585,4.71774
964,"Whs-Brr-Drn-DragonPurple,Scn-Brr-War-Olaf,Tet-Brr-Led-Ornn,Whs-Asn-Pyke,Tet-Asn-Qiyana,Rag-Brr-War-Shen,Whs-Mae-Brr-Sylas,Rel-Brr-TahmKench,Whs-Gun-Thresh",1,1.000000,0,2585,4.71774
10469,"Whs-Brr-Drn-DragonPurple,Rag-She-Asn-Kayn,Tet-Brr-Led-Ornn,Whs-Asn-Pyke,Tet-Asn-Qiyana,Rag-Brr-War-Shen,Whs-Mae-Brr-Sylas",1,8.000000,0,2585,4.71774
9063,"Whs-Brr-Drn-DragonPurple,Whs-Shr-Elise,Trr-Mae-Heimerdinger,Jae-Shr-Neeko,Rag-Brr-War-Shen,Whs-Mae-Brr-Sylas,Mie-Drr-War-Yasuo",1,7.000000,0,2585,4.71774
10468,"Whs-Brr-Drn-DragonPurple,Rag-She-Asn-Kayn,Tet-Brr-Led-Ornn,Whs-Asn-Pyke,Tet-Asn-Qiyana,Rag-Brr-War-Shen,Jae-Str-Soraka,Whs-Mae-Brr-Sylas",1,8.000000,0,2585,4.71774
5834,"Whs-Brr-Drn-DragonPurple,Tet-Brr-Led-Ornn,Whs-Asn-Pyke,Tet-Asn-Qiyana,Rag-Brr-War-Shen,Whs-Mae-Brr-Sylas,Whs-Gun-Thresh",121,4.876033,0,2585,4.71774
9065,"Whs-Brr-Drn-DragonPurple,Whs-Shr-Elise,Trr-Mae-Heimerdinger,Asl-Brr-Illaoi,Tet-Brr-Led-Ornn,Whs-Mae-Brr-Sylas,Whs-Gun-Thresh",1,7.000000,0,2585,4.71774


In [62]:
top5_comp_ranking_list = []
m = comp_ranking_df[comp_ranking_df['grp_count']>=10] #[:5] #Top 5 with counts >= 12
top5_comp_ranking_list.extend(m.values)
top_comp_ranking_df = pd.DataFrame(top5_comp_ranking_list, columns=['comp', 'value_count', 'average_placement', 'group', 'grp_count', 'grp_placement'])

In [63]:
def remove_traits(units_str):
    """Remove units traits from text seperated by comma

    Args:
        units_str (str): traits-unit,traits-unit

    Returns:
        str: Units stripped of traits
    """    
    if len(units_str) == 0:
        return ''

    units_array = []
    for unit in units_str.split(','):
        units_array.append(unit.split('-')[-1])
    units = ', '.join(units_array)
    return units

top_comp_ranking_df['comp'] = top_comp_ranking_df['comp'].apply(remove_traits)


In [64]:
top_comp_ranking_df['mode'] = top_comp_ranking_df.groupby('group')['comp'].transform(lambda x: pd.Series.mode(x)[0])
top_comp_ranking_df.groupby(['group']).head(1).sort_values(by='grp_placement')

,comp,value_count,average_placement,group,grp_count,grp_placement,mode
55,"Bard, Heimerdinger, Illaoi, Lulu, Nami, Sona, Sylas, TrainerDragon, Twitch, Varus, Zoe",1,1.000000,25,11,1.638889,"Bard, Heimerdinger, Illaoi, Lulu, Nami, Sona, Sylas, TrainerDragon, Twitch, Varus"
625,"AoShin, Bard, DragonGold, Lulu, Neeko, Ornn, Soraka",1,1.000000,254,18,1.833333,"AoShin, Bard, DragonGold, DragonGreen, Ornn"
660,"Gnar, Neeko, Sett, Shyvana, Soraka, Swain, Xayah, Yasuo",2,1.000000,276,16,2.659091,"Gnar, Hecarim, Neeko, Sett, Shyvana, Soraka, Swain, Yasuo"
8,"Bard, DragonPurple, Ornn, Qiyana, Sylas, Talon, Yasuo, Zoe",1,1.000000,5,71,2.803030,"Bard, Corki, DragonPurple, Lulu, Sona, Soraka, Sylas, Zoe"
686,"AoShin, Bard, Heimerdinger, Illaoi, Lulu, Nami, Ryze, Sylas, TrainerDragon, Vladimir, Zoe",1,1.000000,289,95,2.853889,"AoShin, Bard, Heimerdinger, Illaoi, LeeSin, Lulu, Sylas, TrainerDragon, Zoe"
...,...,...,...,...,...,...,...
3598,"Heimerdinger, Lillia, Nami, Ryze, Sylas, Varus, Vladimir",1,4.000000,1158,11,6.000000,"Heimerdinger, Illaoi, Lillia, Nami, Ryze, Sejuani, Vladimir"
3255,"Anivia, Heimerdinger, Illaoi, Lulu, Nami, Ryze, Sona, Sylas, TrainerDragon",3,3.666667,988,13,6.074074,"Anivia, AoShin, Heimerdinger, Illaoi, Lulu, Nami, Sylas, TrainerDragon"
473,"Aatrox, DragonGold, Kayn, Neeko, Swain, Volibear, Xayah, Yasuo",1,1.000000,180,11,6.458333,"Aatrox, DragonGold, Hecarim, Kayn, Volibear, Xayah, Yasuo"
4341,"Illaoi, Nami, Nidalee, Skarner, Varus, Vladimir, Xayah",4,4.750000,1304,21,6.683333,"Illaoi, Nami, Nidalee, Pyke, Skarner, Varus, Vladimir"


In [65]:
comp_ranking_df['comp'] = comp_ranking_df['comp'].apply(remove_traits)
comp_ranking_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_comp_ranking.csv'), index=False)

In [66]:
# from statistics import mode
# import jellyfish

# import pandas as pd

# df = pd.DataFrame({'Code': ['abc', 'abc', 'abc', 'abcc', 'abcc', 'zxc'],
#                    'Description': ['ABC String', 'abc string', 'ABC String and sth', 'abc sth else', 'zxc sth else', 'zxc zxc'],
#                 #    'Value': [10, 20, 30, 40, 100]
#                    })

# df_list = []
# for grp,df in df.groupby('Code'):
#     df['distance'] = df['Description'].apply(lambda x : fuzz.token_set_ratio(x, mode(df['Description'])))
#     # df['Description'] =  mode(df['Description'])
#     df_list.append(df[df['distance'] > 10])

# df = pd.concat(df_list)

In [67]:
# comp_ranking_df.groupby('comp')['comp'].apply(lambda x : fuzz.token_set_ratio(x, ','.join(units_col)))

In [68]:
# df_list = []
# for grp,df in comp_ranking_df.groupby('comp'):
#     df['distance'] = df['comp'].apply(lambda x : fuzz.token_set_ratio(x, 'Aatrox, DragonGold, Kayn, Shen, Twitch, Xayah, Zoe'))
#     # df['Description'] =  mode(df['Description'])
#     df_list.append(df[df['distance'] > 10])

# df = pd.concat(df_list)